In [ ]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16353 sha256=94f1a8cd8df8134bd2f4beea0c0b229e40ff5349712e9d00442658d316cbe5fc
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
 

In [ ]:
import pandas as pd
from googletrans import Translator
import random

# Load the dataset
dataset = pd.read_csv('/content/Preprocess_dataset.csv')

# Initialize the translator
translator = Translator()

def back_translate(text, target_lang='en', source_lang='bn'):
    """
    Perform back-translation on the given text.
    """
    try:
        # Translate to target language (English)
        intermediate = translator.translate(text, src=source_lang, dest=target_lang).text
        # Translate back to source language (Bangla)
        result = translator.translate(intermediate, src=target_lang, dest=source_lang).text
        return result
    except Exception as e:
        print(f"Translation error: {e}")
        return text

def generate_new_samples(df, sentiment, target_count):

    sentiment_rows = df[df['sentiment'] == sentiment]
    samples_needed = target_count - len(sentiment_rows)
    new_samples = []

    for _ in range(samples_needed):
        sample = sentiment_rows.sample(n=1).iloc[0]
        new_text = back_translate(sample['conversation_text'])
        new_samples.append({
            'conversation_text': new_text,
            'sentiment': sentiment
        })

    return pd.DataFrame(new_samples)


sentiment_counts = dataset['sentiment'].value_counts()
target_count = sentiment_counts['neutral']  # 45

# Generate new samples
new_positive_samples = generate_new_samples(dataset, 'positive', target_count)
new_negative_samples = generate_new_samples(dataset, 'negative', target_count)

# Combine samples
balanced_dataset = pd.concat([dataset, new_positive_samples, new_negative_samples], ignore_index=True)

# Shuffling dataframe
balanced_dataset = balanced_dataset.sample(frac=1).reset_index(drop=True)

#saving
balanced_dataset.to_csv('balanced_dataset_v1.csv', index=False)

print(balanced_dataset['sentiment'].value_counts())


sentiment
neutral     45
negative    45
positive    45
Name: count, dtype: int64
